In [1]:
import torch 
from torchtext import data, datasets
import random
import torch.nn as nn
import torch.optim as optim
import time

In [2]:
#https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/1%20-%20Simple%20Sentiment%20Analysis.ipynb

In [3]:
#Set your SEED and FIELDs
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnndeterministic=True
TEXT = data.Field(tokenize='spacy', tokenizer_language='en_core_web_sm')
LABEL = data.Field(dtype = torch.float)

In [4]:
%%HTML
<h3> Download Dataset </h3>

In [5]:
#Download Dataset
train_data, test_data = datasets.IMDB.splits(TEXT,LABEL)

In [6]:
#Check shape of test train
len(train_data), len(test_data)

(25000, 25000)

In [7]:
#Check an example
print(vars(train_data.examples[0]))

{'text': ['Bromwell', 'High', 'is', 'a', 'cartoon', 'comedy', '.', 'It', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', ',', 'such', 'as', '"', 'Teachers', '"', '.', 'My', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'Bromwell', 'High', "'s", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"', 'Teachers', '"', '.', 'The', 'scramble', 'to', 'survive', 'financially', ',', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', 'teachers', "'", 'pomp', ',', 'the', 'pettiness', 'of', 'the', 'whole', 'situation', ',', 'all', 'remind', 'me', 'of', 'the', 'schools', 'I', 'knew', 'and', 'their', 'students', '.', 'When', 'I', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school', ',', 'I', 'immediately', 'recalled', '.........', 'at', '..........', 'High', '.', 'A', 'classic', 'l

In [8]:
%%HTML
<h3> Train Test Split </h3>

In [9]:
#Create a validation set with the split() method
#default is 70/30
train_data, val_data = train_data.split(random_state=random.seed(SEED))

In [10]:
#confirm split
len(train_data), len(val_data)

(17500, 7500)

In [11]:
#Now build your Vocabulary or your dict of word to index 
#We're assigning each word an integer value so the ML model may operate
#We will keep top 25,000 words and replace words that dont make the cut iwth an <unk> unkown tag
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [12]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 4


In [13]:
#Note that some entries are padded with a <pad> tag to make entiry inputs the same shape


In [50]:
#Print most common words
print(TEXT.vocab.freqs.most_common(20))

[('the', 203566), (',', 192495), ('.', 165612), ('and', 109442), ('a', 109116), ('of', 100702), ('to', 93766), ('is', 76328), ('in', 61255), ('I', 54004), ('it', 53508), ('that', 49187), ('"', 44282), ("'s", 43329), ('this', 42445), ('-', 36690), ('/><br', 35752), ('was', 35034), ('as', 30384), ('with', 29774)]


In [15]:
#We can also see the vocabulary directly using either the stoi (string to int) or itos (int to string) method

print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [16]:
print(LABEL.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x000001E2E485BFA0>>, {'<unk>': 0, '<pad>': 1, 'neg': 2, 'pos': 3})


In [17]:
#Now create iterators that we will iterate over in the training and val loops and they'll return batches of examples
#These batches will be indexed and converted into tensors
#we will use BucketIterator.splits() to create iters

BATCH_SIZE = 64 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iter, val_iter, test_iter = data.BucketIterator.splits((train_data, val_data,test_data), batch_size = BATCH_SIZE, device = device)

In [18]:
len(train_iter), len(val_iter)

(274, 118)

In [19]:
%%HTML
<h3> Build Model </h3>

In [20]:
#Now Build Model

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, text):
        embedded = self.embedding(text)
        output, hidden = self.rnn(embedded)
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [21]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [22]:
#Create a finction to tell us how many trainable parameters out model has
#You can compare the number of params across different models

def count_params(model):
    '''
    Counts number of paramerters in a given model
    Input:
        model - Pytorch nn
    Output:
        count - number of params
        '''
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_params(model):,} traibable parameters')

The model has 2,592,105 traibable parameters


In [23]:
%%HTML 
<h3> Train Model </h3>

In [24]:
#Create optimizer - SGF
lr = 1e-3
optimizer = optim.SGD(model.parameters(), lr = lr)

In [25]:
#Define Loss function here will use binary cross entropy with logits
#try nLL next

loss_fn = nn.BCEWithLogitsLoss()



In [26]:
#Use GPU if you can

#model= model.to(device)
#loss_fn = loss_fn.to(device)

In [27]:
#Cacluate accuracy

def binary_accuracy(preds, y):
    '''
    Returns accuracy per patch
    Input:
        preds - model predictions
        y - known labels
    '''
    
    #round prediction sto closes integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    
    acc = correct.sum() / len(correct)
    return acc

In [28]:
#define training loop
def train(n_epochs, model, optimizer, loss_fn):
    
    epoch_loss = 0
    epoch_acc = 0
    
    #model.train()
    for epoch in n_epochs:
        optimizer.zero_grad()
        pred = model(epoch.text).squeeze(1)
        pred = pred.unsqueeze(0)
        loss = loss_fn(pred, epoch.label)
        
        acc = binary_accuracy(pred, epoch.label)
            
        
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss/len(n_epochs), epoch_acc/len(n_epochs)

In [29]:
#Make an an evaluate function using model .eval() whihc turns of dropout an dbatch normalization
#Similar to train functino


def eval(n_epochs,model, loss_fn):
    epoch_loss = 0 
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        
        for epoch in n_epochs:
            pred = model(epoch.text).squeeze(1)
            pred = pred.unsqueeze(0)
            loss = loss_fn(pred, epoch.label)
            
            
            acc = binary_accuracy(pred, epoch.label)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
    return epoch_loss/len(n_epochs), epoch_acc/len(n_epochs)

In [30]:
#Now create function to compare epoch lenths between models

def epoch_time(start_time, end_time):
    elapsed_time = end_time -start_time
    elapsed_mins = int( elapsed_time / 60)
    elapsed_secs =int(elapsed_time - (elapsed_mins*60))
    return elapsed_mins, elapsed_secs

In [ ]:
#Save the model paramters that five the best vlaidation loss over each epoch

N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(train_iter,model, optimizer, loss_fn)
    valid_loss, valid_acc = eval(val_iter,model, loss_fn)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [31]:
%%HTML
<h3> Building Long Short-Term Memory LSTM RNN Model </h3>


In [51]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache\glove.6B.zip: 862MB [06:37, 2.17MB/s]                                          
100%|█████████████████████████████████████████████▉| 399999/400000 [00:49<00:00, 8104.61it/s]


In [53]:
#LSTM overcomes the vanishing gradient problem
#We will also implemebt a Bidirection RNN analyzing senteses forward and backward
#Will add Multi-layer RNN
#and Regularization

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
                 bidirectiona, dropout, pad_index):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_index)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers = n_layers, bidirectional = True, dropout = dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self,text,text_lenghts):
        embedding = self. dropout(self.embedding(text))
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded,text_lengths)
        
        packed_output, (hidden,cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths= nn.utlis.rnn.pad_packed_sequence(packed_output)
        
        hidden =self.dropout(torch.cat((hiffen[-2,:,:], hidden[-1,:,:]), dim=1))
        
        return self.fc(hidden)

In [54]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [55]:

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,810,857 trainable parameters


In [56]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [57]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 1.5532, -0.8336, -0.7755,  ...,  1.0340,  0.8363, -0.3129],
        [ 0.7328,  1.8789, -0.8945,  ..., -1.7670,  0.0801, -0.3380],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.9473, -0.3294, -0.0121,  ..., -0.7094,  0.3989,  0.1043],
        [-0.8086,  0.1864,  0.6274,  ..., -0.5705,  0.3693,  0.0856],
        [ 0.3276,  0.0985, -0.3828,  ..., -0.3682,  0.1669, -0.1790]])

In [58]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.9473, -0.3294, -0.0121,  ..., -0.7094,  0.3989,  0.1043],
        [-0.8086,  0.1864,  0.6274,  ..., -0.5705,  0.3693,  0.0856],
        [ 0.3276,  0.0985, -0.3828,  ..., -0.3682,  0.1669, -0.1790]])


In [59]:
optimizer = optim.Adam(model.parameters())

In [60]:
loss_fn = nn.BCEWithLogitsLoss()

model = model.to(device)
loss_fn= loss_fn.to(device)

In [ ]:
#Save the model paramters that five the best vlaidation loss over each epoch

N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(train_iter,model, optimizer, loss_fn)
    valid_loss, valid_acc = eval(val_iter,model, loss_fn)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')